In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

import tensorflow as tf
import tensorflow_fold as td
import cPickle
import config
import numpy as np
from nltk.tokenize import sexpr
from utils import *

sess = tf.InteractiveSession()

test_str = '(2 (3 (3 Effective) (2 but)) (1 (1 too-tepid) (2 biopic)))'
print(tokenize(test_str))
print('\n'*2)
print_senti_tree(test_str)

def block_info(block):
    print("%s: %s -> %s" % (block, block.input_type, block.output_type))

# Basic test

In [2]:
temp = td.Map(td.Scalar())
block_info(temp)

temp.eval([1,2,3])

<td.Map element_block=<td.Scalar dtype='float32'>>: None -> SequenceType(TensorType((), 'float32'))


[array(1.0, dtype=float32),
 array(2.0, dtype=float32),
 array(3.0, dtype=float32)]

In [3]:
temp = (td.Map(td.Scalar()), td.Map(td.Scalar())) >> td.Identity()
block_info(temp)

temp.eval(([1,2,3],[4,5,6]))

<td.Pipe>: None -> TupleType(SequenceType(TensorType((), 'float32')), SequenceType(TensorType((), 'float32')))


([array(1.0, dtype=float32),
  array(2.0, dtype=float32),
  array(3.0, dtype=float32)],
 [array(4.0, dtype=float32),
  array(5.0, dtype=float32),
  array(6.0, dtype=float32)])

In [4]:
temp = (td.Vector(3), td.Vector(3))  >> td.Concat()
block_info(temp)

temp.eval(([1,2,3],[4,5,6]))

<td.Pipe>: PyObjectType() -> TensorType((6,), 'float32')


array([ 1.,  2.,  3.,  4.,  5.,  6.], dtype=float32)

In [5]:
c = td.Composition()
with c.scope():
    x = c.input[0]
    c.output.reads(x)
                  
((td.Scalar(), td.Scalar()) >> c).eval((1,2))

array(1.0, dtype=float32)

In [6]:
c = td.Composition()
with c.scope():
    x = c.input
    c.output.reads(x)
                  
(td.Scalar() >> c).eval(1)

array(1.0, dtype=float32)

# Basic tuple test

In [7]:
c = td.Composition()
with c.scope():
    x = td.Vector(3).reads(c.input)
    x_squared = td.Function(tf.multiply).reads(x, x)
    ten = td.FromTensor(10 * np.ones(3, dtype='float32'))
    ten_x = td.Function(tf.multiply).reads(ten, x)
    c.output.reads(td.Function(tf.add).reads(x_squared, ten_x))
                   
c.eval([1,2,3])

array([ 11.,  24.,  39.], dtype=float32)

In [8]:
pair = td.Composition()
with pair.scope():
    a = td.Function(lambda x: x + 100).reads(pair.input[0])
    b = td.Function(lambda x: x + 200).reads(pair.input[1])
    pair.output.reads(b, a)
    
((td.Scalar(), td.Scalar()) >> pair).eval((1, 2))

(array(202.0, dtype=float32), array(101.0, dtype=float32))

In [9]:
pair = td.AllOf(td.Map(td.Scalar()) >> td.GetItem(1), td.Map(td.Scalar()) >> td.GetItem(0))
pair.eval((1,2))

(array(2.0, dtype=float32), array(1.0, dtype=float32))

In [10]:
pair = td.AllOf(td.Map(td.Scalar()) >> td.GetItem(0) >> td.Function(lambda x: x + 10),
                td.Map(td.Scalar()) >> td.GetItem(0) >> td.Function(lambda x: x + 20))
    
pair.eval((1,2))

(array(11.0, dtype=float32), array(21.0, dtype=float32))

In [11]:
pair = td.AllOf(td.GetItem(1), td.GetItem(0)) >> td.Concat()
((td.Scalar(), td.Scalar()) >> pair).eval((1,2))

array([ 2.,  1.], dtype=float32)

# Concat test

In [12]:
def get_c1():
    return (td.Scalar(), td.Scalar()) >> td.Concat()

c1 = get_c1()
print c1.eval((1,2))

c2 = (td.Scalar(), c1) >> td.Concat()
print c2.eval((3,(1,2)))

c3 = (td.Scalar(), td.Scalar(), td.Scalar()) >> td.Concat()
print c3.eval((1,2,3))

c4 = (get_c1(), get_c1()) >> td.Concat()
print c4.eval(((2,3),(4,5)))

c5 = (td.Scalar(), get_c1(), get_c1()) >> td.Concat()
print c5.eval((1,(2,3),(4,5)))

[ 1.  2.]
[ 3.  1.  2.]
[ 1.  2.  3.]
[ 2.  3.  4.  5.]
[ 1.  2.  3.  4.  5.]


In [13]:
v_block = td.Vector(3)
block_info(v_block)
print v_block.eval([7,8,9])

temp = (td.Scalar(), v_block) >> td.Identity()
block_info(temp)
print temp.eval((1, [7,8,9]))

temp = temp >> td.Concat()
block_info(temp)
print temp.eval((1, [7,8,9]))

print ((td.Vector(3), td.Vector(3)) >> td.Concat()).eval(([1,2,3], [7,8,9]))

<td.Vector dtype='float32' size=3>: PyObjectType() -> TensorType((3,), 'float32')
[ 7.  8.  9.]
<td.Pipe>: PyObjectType() -> TupleType(TensorType((), 'float32'), TensorType((3,), 'float32'))
(array(1.0, dtype=float32), array([ 7.,  8.,  9.], dtype=float32))
<td.Pipe>: PyObjectType() -> TensorType((4,), 'float32')
[ 1.  7.  8.  9.]
[ 1.  2.  3.  7.  8.  9.]


# Label a s-exp with node ids

In [14]:
word_case = td.Composition()
with word_case.scope():
    node_id = word_case.input[0]
    label = (td.InputTransform(tokenize) >> td.GetItem(0)).reads(word_case.input[1])
    s = (td.InputTransform(tokenize) >> td.GetItem(1)).reads(word_case.input[1])
    left_s = td.GetItem(0).reads(s)
    word_case.output.reads(node_id, label, left_s)
((td.Scalar(), td.Identity()) >> word_case).eval((1, test_str))

(array(1.0, dtype=float32), '2', '(3 (3 Effective) (2 but))')

In [15]:
temp = td.GetItem(1) >> td.InputTransform(tokenize) >> td.GetItem(1) >> td.OneOf(len, 
                                                                                 [(1, td.Identity()), (2, td.Identity())])
print ((td.Scalar(), td.Identity()) >> temp).eval((1, test_str))

temp = td.OneOf(td.GetItem(1) >> td.InputTransform(tokenize) >> td.GetItem(1) >> td.InputTransform(len), 
                [(1, td.Identity()), (2, td.Identity())])
print ((td.Scalar(), td.Identity()) >> temp).eval((1, test_str))

['(3 (3 Effective) (2 but))', '(1 (1 too-tepid) (2 biopic))']
(array(1.0, dtype=float32), '(2 (3 (3 Effective) (2 but)) (1 (1 too-tepid) (2 biopic)))')


In [16]:
temp = td.OneOf(lambda x: len(tokenize(x[1])[1]), 
                         {1: (td.InputTransform(lambda x: [x]) >> td.Map(td.Scalar()), td.Identity()) >> td.Identity(), 
                          2: (td.InputTransform(lambda x: [x]) >> td.Map(td.Scalar()), td.InputTransform(tokenize))})

block_info(temp)
(temp).eval((1, test_str))

<td.OneOf>: PyObjectType() -> TupleType(SequenceType(TensorType((), 'float32')), PyObjectType())


([array(1.0, dtype=float32)],
 ('2', ['(3 (3 Effective) (2 but))', '(1 (1 too-tepid) (2 biopic))']))

In [17]:
def embed_tree():
    tree_case = td.OneOf(td.GetItem(1) >> td.InputTransform(lambda x: len(tokenize(x)[1])), 
                         {1: (td.Scalar(), td.Identity()), 2: (td.Scalar(), td.Identity())})
    return  tree_case

model = embed_tree()
block_info(model)

(model).eval((1, test_str))

<td.OneOf>: None -> None


(array(1.0, dtype=float32),
 '(2 (3 (3 Effective) (2 but)) (1 (1 too-tepid) (2 biopic)))')

In [18]:
embed_subtree = td.ForwardDeclaration(name='embed_subtree')

def add_metrics():
    c = td.Composition()
    with c.scope():
        node_id = c.input
        
        td.Metric('node_ids').reads(node_id)
        
        c.output.reads(node_id)
    return c

block_info(add_metrics())

def embed_tree():
    """Input: node_id, s-exp
       Output: node_ids"""
    
    word_case = td.GetItem(0)
    block_info(word_case)
    
    pair_case = td.AllOf(td.Identity(),
                         (td.Function(lambda x: 2*x), 
                          td.InputTransform(tokenize) >> td.InputTransform(lambda x: x[1][0])) 
                             >> embed_subtree(),
                         (td.Function(lambda x: 2*x+1), 
                          td.InputTransform(tokenize) >> td.InputTransform(lambda x: x[1][1])) 
                             >> embed_subtree()
                         ) >> td.GetItem(0) >> td.GetItem(0)
    block_info(pair_case)
    
    tree_case = td.OneOf(td.GetItem(1) >> td.InputTransform(lambda x: len(tokenize(x)[1])), 
                         {1: word_case, 2: pair_case})
    block_info(tree_case)
    
    return tree_case >> add_metrics()


model = (td.Scalar(), td.Identity()) >> embed_tree()

embed_subtree.resolve_to(embed_tree())
block_info(model)                 

#model.eval((1, test_str))

compiler = td.Compiler.create(model)
print('input type: %s' % model.input_type)
print('output type: %s' % model.output_type)

metrics = compiler.metric_tensors['node_ids']

res = sess.run(metrics,
         compiler.build_feed_dict([(1, test_str)]))

print(res)

<td.Composition>: None -> None
<td.GetItem key=0>: None -> None
<td.Pipe>: None -> None
<td.OneOf>: None -> None
<td.GetItem key=0>: None -> None
<td.Pipe>: None -> None
<td.OneOf>: None -> None
<td.Pipe>: None -> None
input type: PyObjectType()
output type: TensorType((), 'float32')
[ 4.  5.  2.  6.  7.  3.  1.]


In [19]:
embed_subtree = td.ForwardDeclaration(name='embed_subtree')

def add_metrics():
    c = td.Composition()
    with c.scope():
        node_id = c.input
        
        td.Metric('node_ids').reads(node_id)
        
        c.output.reads(node_id)
    return c

block_info(add_metrics())

def embed_tree():
    """Input: node_id, s-exp
       Output: node_ids"""
    
    word_case = td.GetItem(0)
    block_info(word_case)
    
    pair_case = td.Composition()
    with pair_case.scope():
        node_id = pair_case.input[0]
        left_node_id = td.Function(lambda x: 2*x).reads(node_id)
        right_node_id = td.Function(lambda x: 2*x+1).reads(node_id)
        tree_s = pair_case.input[1]
        tok_res = td.InputTransform(tokenize).reads(tree_s)
        left_tree_s = (td.GetItem(1) >> td.GetItem(0)).reads(tok_res)
        right_tree_s = (td.GetItem(1) >> td.GetItem(1)).reads(tok_res)
        left_ret = embed_subtree().reads(left_node_id, left_tree_s)
        right_ret = embed_subtree().reads(right_node_id, right_tree_s)
        pair_case.output.reads(node_id, left_ret, right_ret)
        
    block_info(pair_case)
    
    tree_case = td.OneOf(td.GetItem(1) >> td.InputTransform(lambda x: len(tokenize(x)[1])), 
                         {1: word_case, 2: pair_case >> td.GetItem(0)})
    block_info(tree_case)
    
    return tree_case >> add_metrics()


model = (td.Scalar(), td.Identity()) >> embed_tree()

embed_subtree.resolve_to(embed_tree())
block_info(model)                 

#model.eval((1, test_str))

compiler = td.Compiler.create(model)
print('input type: %s' % model.input_type)
print('output type: %s' % model.output_type)

metrics = compiler.metric_tensors['node_ids']

res = sess.run(metrics,
         compiler.build_feed_dict([(1, test_str)]))

print(res)

<td.Composition>: None -> None
<td.GetItem key=0>: None -> None
<td.Composition>: None -> None
<td.OneOf>: None -> None
<td.GetItem key=0>: None -> None
<td.Composition>: None -> None
<td.OneOf>: None -> None
<td.Pipe>: None -> None
input type: PyObjectType()
output type: TensorType((), 'float32')
[ 4.  5.  2.  6.  7.  3.  1.]


In [ ]:
embed_subtree = td.ForwardDeclaration(name='embed_subtree')

class BinaryTreeLSTMCell(tf.contrib.rnn.BasicLSTMCell):
    def __init__(self, num_units, keep_prob=1.0):
        super(BinaryTreeLSTMCell, self).__init__(num_units)
        self._keep_prob = keep_prob

    def __call__(self, inputs, state, scope=None):
        with tf.variable_scope(scope or type(self).__name__):
            lhs, rhs = state
            c0, h0 = lhs
            c1, h1 = rhs
            concat = tf.contrib.layers.linear(
                tf.concat([inputs, h0, h1], 1), 5 * self._num_units)

        # i = input_gate, j = new_input, f = forget_gate, o = output_gate
        i, j, f0, f1, o = tf.split(value=concat, num_or_size_splits=5, axis=1)

        j = self._activation(j)
        if not isinstance(self._keep_prob, float) or self._keep_prob < 1:
            j = tf.nn.dropout(j, self._keep_prob)

        new_c = (c0 * tf.sigmoid(f0 + self._forget_bias) + \
                 c1 * tf.sigmoid(f1 + self._forget_bias) + \
                 tf.sigmoid(i) * j)
        new_h = self._activation(new_c) * tf.sigmoid(o)

        new_state = tf.contrib.rnn.LSTMStateTuple(new_c, new_h)

        return new_h, new_state
    

NUM_CLASSES = 5  # number of distinct sentiment labels
embed_size = 7
state_size = 5
tree_lstm = td.ScopedLayer(
      tf.contrib.rnn.DropoutWrapper(
          BinaryTreeLSTMCell(state_size)),
      name_or_scope='tree_lstm')

output_layer = td.FC(NUM_CLASSES, activation=None, name='output_layer')

def tf_node_loss(logits, labels):
    return tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)

def tf_fine_grained_hits(logits, labels):
    predictions = tf.cast(tf.argmax(logits, 1), tf.int32)
    return tf.cast(tf.equal(predictions, labels), tf.float64)

def tf_binary_hits(logits, labels):
    softmax = tf.nn.softmax(logits)
    binary_predictions = (softmax[:, 3] + softmax[:, 4]) > (softmax[:, 0] + softmax[:, 1])
    binary_labels = labels > 2
    return tf.cast(tf.equal(binary_predictions, binary_labels), tf.float64)

def add_metrics():
    """A block that adds metrics for loss and hits; output is the LSTM state."""
    c = td.Composition()
    with c.scope():
        # destructure the input; (id, (logits, state))
        node_id = c.input[0]
        label = c.input[1]
        
        logits = td.GetItem(0).reads(c.input[2])
        state = td.GetItem(1).reads(c.input[2])
        
        td.Metric('node_ids').reads(node_id)
        td.Metric('logits').reads(logits)
        td.Metric('labels').reads(label)

        # output the state, which will be read by our by parent's LSTM cell
        c.output.reads(state)
    return c

def logits_and_state():
    """Creates a block that goes from (node_id, tree) to (node_id, label, (logits, state)) tuples."""

    pair2vec = td.Composition()
    with pair2vec.scope():
        node_id = pair2vec.input[0]
        left_node_id = td.Function(lambda x: 2*x).reads(node_id)
        right_node_id = td.Function(lambda x: 2*x+1).reads(node_id)
        tree_s = pair2vec.input[1]
        tok_res = td.InputTransform(tokenize).reads(tree_s)
        left_tree_s = (td.GetItem(1) >> td.GetItem(0)).reads(tok_res)
        right_tree_s = (td.GetItem(1) >> td.GetItem(1)).reads(tok_res)
        left_ret = embed_subtree().reads(left_node_id, left_tree_s)
        right_ret = embed_subtree().reads(right_node_id, right_tree_s)
        pair2vec.output.reads(left_ret, right_ret)
    block_info(pair2vec)

    
    word_case = td.AllOf(td.FromTensor(np.random.rand(embed_size).astype(np.float32)), td.Zeros((tree_lstm.state_size,) * 2))
    block_info(word_case)
    
    pair_case = td.AllOf(td.Zeros(embed_size), pair2vec)
    block_info(pair_case)
    
    tree2vec = td.OneOf(td.GetItem(1) >> td.InputTransform(lambda x: len(tokenize(x)[1])), [(1, word_case), (2, pair_case)])
    block_info(tree2vec)
    
    res = td.AllOf(td.GetItem(0), # node_id
                   td.GetItem(1) >> td.InputTransform(lambda x: tokenize(x)[0]) >> td.Scalar('int32'),
                   tree2vec >> tree_lstm >> (output_layer, td.Identity())
                  )
    return res

def embed_tree(logits_and_state, is_root):
    """Creates a block that embeds (node_id, trees); output is tree LSTM state."""
    return logits_and_state >> add_metrics()

model = (td.Scalar('int32'), td.Identity()) >> embed_tree(logits_and_state(), is_root=True)

embed_subtree.resolve_to(embed_tree(logits_and_state(), is_root=False))
block_info(model)                 


compiler = td.Compiler.create(model)
print('input type: %s' % model.input_type)
print('output type: %s' % model.output_type)

metrics = compiler.metric_tensors['node_ids']
logits = compiler.metric_tensors['logits']
labels = compiler.metric_tensors['labels']

print(metrics)
print(logits)
print(labels)

sess.run(tf.global_variables_initializer())
res = sess.run([metrics, logits, labels],
         compiler.build_feed_dict([(1, test_str)]))

print(res)
print('\n'*2)
print(res[0])
print(type(res[0]))
print(type(res[2]))
print_senti_tree(test_str)